# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,City,Country,Lat,Lng,Date,Max Temp,Cloudiness,Wind Speed,Humidity
0,georgetown,MY,5.41,100.34,1587876955,89.60,20,2.24,79
1,rikitea,PF,-23.12,-134.97,1587877079,76.01,0,17.31,76
2,punta arenas,CL,-53.15,-70.92,1587876867,30.20,0,3.36,80
3,jamestown,US,42.10,-79.24,1587876993,51.80,90,10.29,87
4,mataura,NZ,-46.19,168.86,1587877079,61.00,95,13.00,57
...,...,...,...,...,...,...,...,...,...
557,manaquiri,BR,-3.32,-60.35,1587877181,75.20,40,2.24,94
558,chumikan,RU,54.72,135.31,1587877181,38.01,93,3.09,83
559,mayumba,GA,-3.43,10.66,1587877181,77.09,100,6.80,87
560,duluth,US,46.78,-92.11,1587877181,39.99,90,12.75,86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
fig = gmaps.figure()

#Set locations for the heat map. 
locations = weather_data[["Lat", "Lng"]]

#Set humidity as weight for the heat map. 
humidity = weather_data["Humidity"]

In [7]:

#Create heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, max_intensity=10, 
                                point_radius=1, dissipating=False)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
new_weather_data = weather_data.loc[(weather_data["Cloudiness"] == 0) \
                                   & (weather_data["Humidity"]< 65) \
                                   & (weather_data["Max Temp"]>70) & (weather_data["Max Temp"]<80) \
                                   & (weather_data["Wind Speed"]< 10)]
new_weather_data.dropna()

,City,Country,Lat,Lng,Date,Max Temp,Cloudiness,Wind Speed,Humidity
89,culiacan,MX,24.80,-107.39,1587876885,73.00,0,1.92,62
147,puerto penasco,MX,31.32,-113.53,1587877105,79.00,0,5.99,51
238,korla,CN,41.76,86.15,1587877135,77.13,0,1.70,11
255,taoudenni,ML,22.68,-3.98,1587877140,75.69,0,6.46,18
267,aswan,EG,24.09,32.91,1587877141,77.36,0,8.68,27
459,gondar,ET,12.60,37.47,1587877167,75.31,0,4.81,20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = new_weather_data.filter(["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
89,culiacan,MX,24.80,-107.39,
147,puerto penasco,MX,31.32,-113.53,
238,korla,CN,41.76,86.15,
255,taoudenni,ML,22.68,-3.98,
267,aswan,EG,24.09,32.91,
459,gondar,ET,12.60,37.47,


In [31]:
#Set params 
params = {"radius": 5000, 
         "types": "lodging",
         "key": "g_key"}

#Loop through hotel_df and find hotels in each city: 
for index, row in hotel_df.iterrows(): 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Change locations while keeping the params same
    params["location"] = f"{lat},{lng}"
    
    #make request and print url
    name_address = requests.get(base_url, params=params).json()
  
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping")
    


Missing field/result... skipping
Missing field/result... skipping
Missing field/result... skipping
Missing field/result... skipping
Missing field/result... skipping
Missing field/result... skipping


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))